In [1]:
import requests
import pandas as pd
from youtube_comment_downloader import YoutubeCommentDownloader
from itertools import islice
import time
import json
import random
import os

# ===========================
# CONFIGURACIÓN GENERAL
# ===========================

OUTPUT_FILE = "data/redes_sociales/corpus_genz.txt"
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

TOTAL_OBJETIVO = 5000

# ======== YOUTUBE ========
LIMIT_PER_VIDEO = 300  # puedes subirlo a 500 si quieres un dataset más grande
VIDEO_URLS = [
    "https://www.youtube.com/watch?v=QA9Wfyh0mFQ", # Why Gen Z is so Lonely
    "https://www.youtube.com/watch?v=x7-sVvW2-5Y", # La sociedad del cansancio
    "https://www.youtube.com/watch?v=d_2yCIDt7m8", # I quit social media for 30 days
    "https://www.youtube.com/watch?v=ZKk5sJ6S5rY", # Social media is destroying your brain
    "https://www.youtube.com/watch?v=0ggK1Qz7HnI", # Digital burnout explained
]

# ======== REDDIT ========
# Lista de comunidades relevantes
SUBREDDITS = [
    "GenZ",
    "InternetAddiction",
    "depression",
    "socialmedia",
    "digitalminimalism",
    "mentalhealth",
    "offmychest",
    "lonely",
    "burnout",
]

# Términos temáticos para filtrar
QUERY_TERMS = [
    "burnout",
    "lonely",
    "dopamine detox",
    "quit social media",
    "meaning crisis",
    "overwhelmed",
    "anxiety",
    "existential",
    "digital overload",
    "identity",
    "tired of life",
]

# ===========================
# FUNCIONES AUXILIARES
# ===========================

def limpiar_texto(t):
    if not t:
        return ""
    t = t.replace("\n", " ").strip()
    t = " ".join(t.split())  # quitar espacios múltiples
    return t


# ===========================
# SCRAPER YOUTUBE
# ===========================

def scrape_youtube():
    print("\n=== Descargando datos de YouTube ===")
    downloader = YoutubeCommentDownloader()
    comentarios = []

    for url in VIDEO_URLS:
        print(f"> Procesando video: {url}")
        try:
            gen = downloader.get_comments_from_url(url, sort_by=1)
            for comment in islice(gen, LIMIT_PER_VIDEO):
                texto = limpiar_texto(comment.get("text", ""))
                if len(texto) > 30:
                    comentarios.append(
                        f"FUENTE:YOUTUBE | URL:{url} | COMENTARIO:{texto}"
                    )
        except Exception as e:
            print(f"Error en {url}: {e}")

    print(f"Total YouTube: {len(comentarios)}\n")
    return comentarios


# ===========================
# SCRAPER REDDIT (Pushshift)
# ===========================

def pushshift_request(endpoint, params):
    """Pequeño wrapper para manejar errores y reintentos."""
    url = f"https://api.pushshift.io/reddit/{endpoint}"
    for _ in range(3):
        try:
            r = requests.get(url, params=params, timeout=10)
            if r.status_code == 200:
                return r.json()
        except:
            time.sleep(1)
    return {"data": []}


def scrape_reddit(limit_total=3000):
    print("\n=== Descargando datos de Reddit ===")
    resultados = []

    for sub in SUBREDDITS:
        print(f"> Subreddit: r/{sub}")

        for term in QUERY_TERMS:
            params = {
                "subreddit": sub,
                "q": term,
                "size": 100,  # máximo permitido por request
                "sort": "desc",
            }

            data = pushshift_request("search/comment", params).get("data", [])
            for c in data:
                texto = limpiar_texto(c.get("body", ""))
                if len(texto) > 20 and "http" not in texto and "www" not in texto:
                    resultados.append(
                        f"FUENTE:REDDIT | SUB:{sub} | QUERY:{term} | COMENTARIO:{texto}"
                    )

            time.sleep(0.3)

            if len(resultados) >= limit_total:
                break
        if len(resultados) >= limit_total:
            break

    print(f"Total Reddit: {len(resultados)}\n")
    return resultados


# ===========================
# GENERADOR PRINCIPAL
# ===========================

def generar_corpus():

    youtube_data = scrape_youtube()
    reddit_data = scrape_reddit(limit_total=4500)

    corpus = youtube_data + reddit_data

    random.shuffle(corpus)

    # Podamos al objetivo (suelen ser >5k)
    corpus = corpus[:TOTAL_OBJETIVO]

    # Guardado final
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        for linea in corpus:
            f.write(linea + "\n")

    print("=== COMPLETADO ===")
    print(f"Entradas guardadas: {len(corpus)}")
    print(f"Archivo: {OUTPUT_FILE}")


if __name__ == "__main__":
    generar_corpus()



=== Descargando datos de YouTube ===
> Procesando video: https://www.youtube.com/watch?v=QA9Wfyh0mFQ
> Procesando video: https://www.youtube.com/watch?v=x7-sVvW2-5Y
> Procesando video: https://www.youtube.com/watch?v=d_2yCIDt7m8
> Procesando video: https://www.youtube.com/watch?v=ZKk5sJ6S5rY
> Procesando video: https://www.youtube.com/watch?v=0ggK1Qz7HnI
Total YouTube: 0


=== Descargando datos de Reddit ===
> Subreddit: r/GenZ
> Subreddit: r/InternetAddiction
> Subreddit: r/depression
> Subreddit: r/socialmedia
> Subreddit: r/digitalminimalism
> Subreddit: r/mentalhealth
> Subreddit: r/offmychest
> Subreddit: r/lonely
> Subreddit: r/burnout
Total Reddit: 0

=== COMPLETADO ===
Entradas guardadas: 0
Archivo: data/redes_sociales/corpus_genz.txt


In [2]:
import os
import subprocess
import json
import time
import requests
import random

OUTPUT = "data/redes_sociales/corpus_genz.txt"
os.makedirs(os.path.dirname(OUTPUT), exist_ok=True)

# ============================
# CONFIG
# ============================

VIDEO_URLS = [
    "https://www.youtube.com/watch?v=D4SH409Hb9s",
    "https://www.youtube.com/watch?v=F_8d6UgU6I8",
    "https://www.youtube.com/watch?v=p8B3mYlnIVM",
]

SUBREDDITS = [
    "GenZ", "MentalHealth", "digitalminimalism", "InternetAddiction",
    "offmychest", "lonely", "depression"
]

QUERY_TERMS = [
    "burnout", "lonely", "identity", "dopamine detox",
    "quit social media", "meaning", "overwhelmed"
]


# ============================
# YOUTUBE: usar yt-dlp
# ============================

def scrape_youtube():
    print("\n=== Descargando YouTube con yt-dlp ===")

    comentarios = []

    for url in VIDEO_URLS:
        print(f"> Video: {url}")

        try:
            # Ejecuta yt-dlp y captura JSON de comentarios
            cmd = [
                "yt-dlp",
                "--get-comments",
                "--no-warnings",
                "--dump-json",
                url
            ]
            result = subprocess.run(cmd, capture_output=True, text=True)

            if result.stdout.strip() == "":
                print("  ❌ No se obtuvieron comentarios")
                continue

            for line in result.stdout.splitlines():
                try:
                    data = json.loads(line)
                    text = data.get("text", "").strip()
                    if len(text) > 20:
                        comentarios.append(f"YOUTUBE | {url} | {text}")
                except:
                    pass

            print(f"  → Comentarios extraídos: {len(comentarios)}")

        except Exception as e:
            print(f"  Error: {e}")

    return comentarios


# ============================
# REDDIT: API no documentada
# ============================

def scrape_reddit(limit=2000):
    print("\n=== Descargando Reddit ===")
    todos = []

    for sub in SUBREDDITS:
        for term in QUERY_TERMS:
            print(f"> r/{sub} | '{term}'")

            url = f"https://gateway.reddit.com/desktopapi/v1/subreddits/{sub}/posts"
            params = {
                "q": term,
                "sort": "new",
                "limit": 50
            }

            try:
                r = requests.get(url, params=params, timeout=10, headers={
                    "User-Agent": "Mozilla/5.0"
                })

                data = r.json()

                posts = data.get("posts", {})
                if not posts:
                    print("  ❌ Nada encontrado")
                    continue

                for pid, p in posts.items():
                    body = (p.get("source", {}) or {}).get("content", "")
                    if body and len(body) > 30:
                        todos.append(f"REDDIT | r/{sub} | {term} | {body}")

            except Exception as e:
                print("  Error:", e)

            time.sleep(0.4)

            if len(todos) >= limit:
                return todos
    
    return todos


# ============================
# MAIN
# ============================

def main():

    yt = scrape_youtube()
    rd = scrape_reddit()

    corpus = yt + rd
    random.shuffle(corpus)

    print("\n=== Guardando archivo ===")
    print(f"Total: {len(corpus)} líneas")

    with open(OUTPUT, "w", encoding="utf-8") as f:
        for line in corpus:
            f.write(line + "\n")

    print("Listo:", OUTPUT)


if __name__ == "__main__":
    main()



=== Descargando YouTube con yt-dlp ===
> Video: https://www.youtube.com/watch?v=D4SH409Hb9s
  Error: [WinError 2] El sistema no puede encontrar el archivo especificado
> Video: https://www.youtube.com/watch?v=F_8d6UgU6I8
  Error: [WinError 2] El sistema no puede encontrar el archivo especificado
> Video: https://www.youtube.com/watch?v=p8B3mYlnIVM
  Error: [WinError 2] El sistema no puede encontrar el archivo especificado

=== Descargando Reddit ===
> r/GenZ | 'burnout'
  Error: Expecting value: line 1 column 1 (char 0)
> r/GenZ | 'lonely'
  Error: Expecting value: line 1 column 1 (char 0)
> r/GenZ | 'identity'
  Error: Expecting value: line 1 column 1 (char 0)
> r/GenZ | 'dopamine detox'
  Error: Expecting value: line 1 column 1 (char 0)
> r/GenZ | 'quit social media'
  Error: Expecting value: line 1 column 1 (char 0)
> r/GenZ | 'meaning'
  Error: Expecting value: line 1 column 1 (char 0)
> r/GenZ | 'overwhelmed'
  Error: Expecting value: line 1 column 1 (char 0)
> r/MentalHealth | '

In [4]:
import os
import subprocess
import json
import time
import requests
import random
from bs4 import BeautifulSoup

# =========================================
# CONFIGURACIÓN
# =========================================

OUTPUT = "data/redes_sociales/corpus_genz.txt"
os.makedirs(os.path.dirname(OUTPUT), exist_ok=True)

# Videos activos y relevantes
VIDEO_URLS = [
    "https://www.youtube.com/watch?v=D4SH409Hb9s",
    "https://www.youtube.com/watch?v=F_8d6UgU6I8",
    "https://www.youtube.com/watch?v=p8B3mYlnIVM",
]

# Subreddits para raspar sin API
SUBREDDITS = [
    "GenZ",
    "MentalHealth",
    "digitalminimalism",
    "offmychest",
    "InternetAddiction",
    "lonely",
    "depression"
]


# =========================================
# UTILIDADES
# =========================================

def clean(text):
    if not text:
        return ""
    text = text.replace("\n", " ").strip()
    return " ".join(text.split())


# =========================================
# YouTube scraping con yt-dlp
# =========================================

def determine_ytdlp_command():
    """
    Detecta si yt-dlp está en PATH.
    Si NO, usa: python -m yt_dlp
    """
    try:
        test = subprocess.run(["yt-dlp", "--version"], capture_output=True)
        if test.returncode == 0:
            return ["yt-dlp"]
    except:
        pass

    # fallback para Windows / PIP local
    return ["python", "-m", "yt_dlp"]


def scrape_youtube():
    print("\n=== Descargando YouTube ===")

    yt_cmd = determine_ytdlp_command()
    comentarios = []

    for url in VIDEO_URLS:
        print(f"> Video: {url}")

        cmd = yt_cmd + [
                            "--get-comments",
                            "--extractor-args", "youtube:player_client=android",
                            "--dump-json",
                            "--no-warnings",
                            url
                        ]


        try:
            result = subprocess.run(cmd, capture_output=True, text=True)

            if not result.stdout.strip():
                print("  ❌ No se obtuvieron comentarios")
                continue

            count = 0
            for line in result.stdout.splitlines():
                try:
                    data = json.loads(line)
                    text = clean(data.get("text", ""))
                    if len(text) > 20:
                        comentarios.append(f"YOUTUBE | {url} | {text}")
                        count += 1
                except:
                    pass

            print(f"  → {count} comentarios extraídos")

        except Exception as e:
            print(f"  Error: {e}")

    return comentarios


# =========================================
# Reddit scraping sin API (BeautifulSoup)
# =========================================

def scrape_reddit_post(sub):
    """
    Raspa posts recientes de la subreddit (sin API, vía HTML).
    """
    url = f"https://www.reddit.com/r/{sub}/new/"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code != 200:
            return []

        soup = BeautifulSoup(r.text, "html.parser")
        posts = []

        # h3 suele tener títulos, que suelen reflejar quejas, confesiones o temas de salud mental
        for h3 in soup.find_all("h3"):
            text = clean(h3.get_text())
            if len(text) > 25:
                posts.append(f"REDDIT | r/{sub} | POST | {text}")

        return posts

    except Exception as e:
        print(f"  Error en r/{sub}: {e}")
        return []


def scrape_reddit():
    print("\n=== Descargando Reddit (sin API) ===")

    resultados = []

    for sub in SUBREDDITS:
        print(f"> r/{sub}")

        posts = scrape_reddit_post(sub)
        print(f"  → {len(posts)} posts encontrados")

        resultados.extend(posts)

        # Para no spamear Reddit
        time.sleep(1.2)

    return resultados


# =========================================
# MAIN
# =========================================

def main():

    yt = scrape_youtube()
    rd = scrape_reddit()

    corpus = yt + rd
    random.shuffle(corpus)

    print("\n=== Guardando archivo final ===")
    print(f"Total: {len(corpus)} líneas")

    with open(OUTPUT, "w", encoding="utf-8") as f:
        for line in corpus:
            f.write(line + "\n")

    print("Listo →", OUTPUT)


if __name__ == "__main__":
    main()



=== Descargando YouTube ===
> Video: https://www.youtube.com/watch?v=D4SH409Hb9s
  → 0 comentarios extraídos
> Video: https://www.youtube.com/watch?v=F_8d6UgU6I8
  → 0 comentarios extraídos
> Video: https://www.youtube.com/watch?v=p8B3mYlnIVM
  → 0 comentarios extraídos

=== Descargando Reddit (sin API) ===
> r/GenZ
  → 0 posts encontrados
> r/MentalHealth
  → 0 posts encontrados
> r/digitalminimalism
  → 0 posts encontrados
> r/offmychest
  → 0 posts encontrados
> r/InternetAddiction
  → 0 posts encontrados
> r/lonely
  → 0 posts encontrados
> r/depression
  → 0 posts encontrados

=== Guardando archivo final ===
Total: 0 líneas
Listo → data/redes_sociales/corpus_genz.txt


In [8]:
import pandas as pd
from youtube_comment_downloader import YoutubeCommentDownloader
from itertools import islice

# Configuración
OUTPUT_FILE = "data/redes_sociales/comentarios_genz.txt"
LIMIT_PER_VIDEO = 500  # Cantidad de comentarios por video para ser rápidos

# Videos seleccionados (Temas: Adicción a redes, Soledad, Burnout Gen Z)
# Puedes añadir más URLs aquí si quieres.
# Ejemplo de lista expandida
VIDEO_URLS = [
    # 🎯 Tema: Generación Z - Crisis e identidad
    "https://www.youtube.com/watch?v=ogNq_vBtn6c",   # "Gen Z is Traumatized" :contentReference[oaicite:0]{index=0}
    "https://www.youtube.com/watch?v=mcMuApEmW-M",   # "Gen Z is having an identity crisis" :contentReference[oaicite:1]{index=1}
    "https://www.youtube.com/watch?v=uUfn6YCS9SQ",   # "Gen Z's Identity Crisis" :contentReference[oaicite:2]{index=2}
    "https://www.youtube.com/watch?v=nrVjMAqsdwU",   # "The Gen Z Identity Crisis" :contentReference[oaicite:3]{index=3}
    "https://www.youtube.com/watch?v=fnw6vkquGLc",   # "Gen Z is having an identity crisis. Probably because..." :contentReference[oaicite:4]{index=4}
    "https://www.youtube.com/watch?v=PVMT-lexoqQ",   # "Gen Z Deserves Better" (medio testimonial de fatiga) :contentReference[oaicite:5]{index=5}
    "https://www.youtube.com/watch?v=A_tBbWLXaiA",   # "Can Gen Z break the cycle of trauma?" :contentReference[oaicite:6]{index=6}

    # Incluye también estos videos relacionados con crisis emocional de Gen Z:
    "https://www.youtube.com/watch?v=vm5tGIDUS9E",   # "La Cruda REALIDAD de la GEN Z" (Psico Vlog, crisis de salud mental) :contentReference[oaicite:7]{index=7}
    "https://www.youtube.com/watch?v=Nh9dWEHj9do",   # "Crisis salud mental en la Gen Z" (Infographics Show en español) :contentReference[oaicite:8]{index=8}

    # ✅ Opcional: agrega videos similares desde listas de reproducción,
    # canales estilo "mindset Gen Z" o "análisis generacional" que encuentres manualmente.
    
    # 🤖 Tema: IA, autonomía humana, control tecnológico
    "https://www.youtube.com/watch?v=7JkPWHr7sTY",   # Debate Suleyman & Harari (IA, control, futuro humano) :contentReference[oaicite:9]{index=9}
    "https://www.youtube.com/watch?v=jt3Ul3rPXaE",   # "AI & human evolution" Harari (agente vs herramienta) :contentReference[oaicite:10]{index=10}
    "https://www.youtube.com/watch?v=uuBLxWowDqI",   # Harari en Reid & Aria (confianza y control en IA) :contentReference[oaicite:11]{index=11}
    "https://www.youtube.com/watch?v=0BnZMeFtoAM",   # "AI: how to control alien intelligence?" (autonomía incontrolable) :contentReference[oaicite:12]{index=12}
    "https://www.youtube.com/watch?v=7qhHzQSjwuQ",   # "Redefining Humanity in the Age of ... " (IA vs poder humano) :contentReference[oaicite:13]{index=13}

    # 📌 Importante clásico sobre automatización y pérdida de trabajo:
    "https://www.youtube.com/watch?v=7Pq-S557XQU",   # "Humans Need Not Apply" (CGP Grey, automatización del trabajo humano) :contentReference[oaicite:14]{index=14}

    # ➕ Sugerencia de playlists para expandir:
    "https://www.youtube.com/playlist?list=PLfc2WtGuVPdkhyIrhRmrDQEeZ1WEJsKjo", # Harari future playlist (varios videos IA/autonomía) :contentReference[oaicite:15]{index=15}
    "https://www.youtube.com/playlist?list=PLAq4NNj5qTBsK3kA7rHuilWJdswyuKj4d"  # IA & future of humanity playlist (Harari y otros) :contentReference[oaicite:16]{index=16}
]



def descargar_comentarios():
    downloader = YoutubeCommentDownloader()
    all_comments = []

    print("--- Iniciando descarga de comentarios ---")
    
    for url in VIDEO_URLS:
        print(f"Procesando: {url}")
        try:
            # Obtener comentarios (generador)
            comments = downloader.get_comments_from_url(url, sort_by=1) # sort_by=1 es "Más recientes"
            
            # Tomar solo los primeros N comentarios
            for comment in islice(comments, LIMIT_PER_VIDEO):
                text = comment['text']
                # Limpieza básica rápida: eliminar saltos de línea excesivos
                text_clean = text.replace('\n', ' ').strip()
                
                if len(text_clean) > 30: # Ignorar comentarios muy cortos como "lol"
                    all_comments.append(f"FUENTE: YOUTUBE_VIDEO | COMENTARIO: {text_clean}")
                    
        except Exception as e:
            print(f"Error en video {url}: {e}")

    # Guardar en txt
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        for c in all_comments:
            f.write(c + "\n")
            
    print(f"\n--- Éxito! ---")
    print(f"Se guardaron {len(all_comments)} comentarios en: {OUTPUT_FILE}")

if __name__ == "__main__":
    descargar_comentarios()

--- Iniciando descarga de comentarios ---
Procesando: https://www.youtube.com/watch?v=ogNq_vBtn6c
Procesando: https://www.youtube.com/watch?v=mcMuApEmW-M
Procesando: https://www.youtube.com/watch?v=uUfn6YCS9SQ
Procesando: https://www.youtube.com/watch?v=nrVjMAqsdwU
Procesando: https://www.youtube.com/watch?v=fnw6vkquGLc
Procesando: https://www.youtube.com/watch?v=PVMT-lexoqQ
Procesando: https://www.youtube.com/watch?v=A_tBbWLXaiA
Procesando: https://www.youtube.com/watch?v=vm5tGIDUS9E
Procesando: https://www.youtube.com/watch?v=Nh9dWEHj9do
Procesando: https://www.youtube.com/watch?v=7JkPWHr7sTY
Procesando: https://www.youtube.com/watch?v=jt3Ul3rPXaE
Procesando: https://www.youtube.com/watch?v=uuBLxWowDqI
Procesando: https://www.youtube.com/watch?v=0BnZMeFtoAM
Procesando: https://www.youtube.com/watch?v=7qhHzQSjwuQ
Procesando: https://www.youtube.com/watch?v=7Pq-S557XQU
Procesando: https://www.youtube.com/playlist?list=PLfc2WtGuVPdkhyIrhRmrDQEeZ1WEJsKjo
Procesando: https://www.youtube.c